In [34]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [36]:
import requests
from bs4 import BeautifulSoup

In [37]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(page.text)

In [46]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

import pandas as pd
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [47]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [55]:
df.shape

(103, 3)

Section 1 link: https://github.com/thyleung2019/Segmenting-ClusteringToronto/blob/main/Segmenting%20and%20Clustering%20Neighbourhoods%20in%20Toronto.ipynb

In [28]:
conda install -c conda-forge geocoder

Solving environment: done

## Package Plan ##

  environment location: /Users/timleung/opt/anaconda3

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2



geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: 

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M5A'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

Unable to get the geographical coordinates of neighborhood even only asking for 1 postcode 'M5A'. Therefore will import data from csv provided by Coursera

In [42]:
lat_lng_coord = pd.read_csv('Geospatial_Coordinates.csv')

In [43]:
lat_lng_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [48]:
df_lat_lng = pd.merge(df,lat_lng_coord,on='Postal Code',how='left')

In [56]:
df_lat_lng.shape

(103, 5)

In [57]:
df_lat_lng.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Section 2 github link: https://github.com/thyleung2019/Segmenting-ClusteringToronto/blob/Part-2/Segmenting%20and%20Clustering%20Neighbourhoods%20in%20Toronto2.ipynb

In [58]:
from sklearn.cluster import KMeans

In [62]:
k_means = KMeans(init = "k-means++",n_clusters=5, n_init=12)
data = df_lat_lng.drop(['Postal Code','Borough','Neighborhood'],axis=1)
data

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494
...,...,...
98,43.653654,-79.506944
99,43.665860,-79.383160
100,43.662744,-79.321558
101,43.636258,-79.498509


In [63]:
k_means.fit(data)

KMeans(n_clusters=5, n_init=12)

In [67]:
k_means_cat=k_means.labels_
k_means_cat

array([1, 1, 2, 3, 2, 0, 4, 1, 1, 2, 3, 0, 4, 1, 1, 2, 2, 0, 4, 1, 2, 2,
       4, 2, 2, 2, 4, 3, 3, 1, 2, 2, 4, 1, 3, 1, 2, 2, 1, 3, 3, 2, 2, 2,
       1, 3, 0, 1, 2, 0, 0, 1, 3, 0, 2, 3, 0, 0, 1, 3, 0, 3, 3, 0, 0, 1,
       3, 2, 2, 0, 0, 1, 3, 3, 2, 2, 0, 0, 4, 2, 2, 0, 1, 2, 2, 4, 2, 2,
       0, 0, 1, 2, 2, 0, 0, 4, 2, 2, 0, 2, 1, 0, 0], dtype=int32)

In [72]:
df_lat_lng_Kmeans = df_lat_lng

In [74]:
df_lat_lng_Kmeans.insert(1,'Cluster Group',k_means_cat)

In [75]:
df_lat_lng_Kmeans.head()

,Postal Code,Cluster Group,Borough,Neighborhood,Latitude,Longitude
0,M3A,1,North York,Parkwoods,43.753259,-79.329656
1,M4A,1,North York,Victoria Village,43.725882,-79.315572
2,M5A,2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,2,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [76]:
latitude = df_lat_lng_Kmeans["Latitude"].mean()
longitude = df_lat_lng_Kmeans["Longitude"].mean()

In [81]:
import folium
import numpy as np
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
kclusters=5
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_lat_lng_Kmeans['Latitude'], df_lat_lng_Kmeans['Longitude'], df_lat_lng_Kmeans['Neighborhood'], df_lat_lng_Kmeans['Cluster Group']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters